In [1]:
#conda install -c conda-forge ipywidgets

In [1]:
import pandas as pd
import folium as fo
import re
import bs4 
import requests 
import ipywidgets as widgets

In [2]:
def get_data():  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    pagina = requests.get('http://queimadas.dgi.inpe.br/queimadas/dados-abertos/')
    content = pagina.content
    soup = bs4.BeautifulSoup(content)

    lista_regiao_url = []
    lista_regiao = []
    
    for tr in soup.findAll('tr'):                        
        regiao_url = []        
        regiao_nome = []        
        for td in tr.findAll('td'):                        
            
            regiao_nome.append(td.get_text().strip())                                    
            for a in td.find_all('a'):                
                if (len(regiao_nome)==4):   #dados csv 48h                  
                    regiao_url.append(a.get('href'))                    
            
        lista_regiao_url.append([regiao_nome[0],regiao_url[0]])
        lista_regiao.append(regiao_nome[0])
        
    return lista_regiao_url,  lista_regiao
    
lista = get_data()

In [16]:
lista

([['América do Sul',
   '/queimadas/dados-abertos/download/?utm_campaign=dados-abertos&outputFormat=csv&utm_medium=landing-page&time=48h&utm_content=focos_ams_48h&id=focos_ams&utm_source=landing-page'],
  ['Brasil',
   '/queimadas/dados-abertos/download/?utm_campaign=dados-abertos&outputFormat=csv&utm_medium=landing-page&time=48h&utm_content=focos_brasil_48h&id=focos_brasil&utm_source=landing-page'],
  ['Acre',
   '/queimadas/dados-abertos/download/?utm_campaign=dados-abertos&outputFormat=csv&utm_medium=landing-page&time=48h&utm_content=focos_brasil_ac_48h&id=focos_brasil_ac&utm_source=landing-page'],
  ['Alagoas',
   '/queimadas/dados-abertos/download/?utm_campaign=dados-abertos&outputFormat=csv&utm_medium=landing-page&time=48h&utm_content=focos_brasil_al_48h&id=focos_brasil_al&utm_source=landing-page'],
  ['Amapá',
   '/queimadas/dados-abertos/download/?utm_campaign=dados-abertos&outputFormat=csv&utm_medium=landing-page&time=48h&utm_content=focos_brasil_ap_48h&id=focos_brasil_ap&utm_

In [3]:
print('Selecione a regiao que deseja inspecionar os focos de incendio:')
ToggleButtons_regiao = widgets.ToggleButtons(
    options=lista[1],    
    value=lista[1][1],    
    disabled=False    
)

display(ToggleButtons_regiao)

Selecione a regiao que deseja inspecionar os focos de incendio:


ToggleButtons(index=1, options=('América do Sul', 'Brasil', 'Acre', 'Alagoas', 'Amapá', 'Amazonas', 'Bahia', '…

In [4]:
idx = lista[1].index(ToggleButtons_regiao.value)

In [5]:
print(idx)

14


In [6]:
url = 'http://queimadas.dgi.inpe.br' +  str(lista[0][idx][1])
print(url)

http://queimadas.dgi.inpe.br/queimadas/dados-abertos/download/?utm_campaign=dados-abertos&outputFormat=csv&utm_medium=landing-page&time=48h&utm_content=focos_brasil_mg_48h&id=focos_brasil_mg&utm_source=landing-page


In [7]:
CSV_URL = url
#'http://queimadas.dgi.inpe.br/queimadas/dados-abertos/download/?utm_campaign=dados-abertos&outputFormat=csv&utm_medium=landing-page&time=48h&utm_content=focos_brasil_48h&id=focos_brasil&utm_source=landing-page'

In [8]:
df = pd.read_csv(CSV_URL)

In [9]:
df.head()

,FID,id,longitude,latitude,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,geom
0,focos_brasil_ontem_hoje_ref.e66538e3-f99d-319d...,e66538e3-f99d-319d-8287-46301be952f7,-49.616,-18.624,2020-09-25T17:10:00,AQUA_M-T,CAPINÓPOLIS,MINAS GERAIS,Brasil,3112604,31,33,2,0.0,1.0,Cerrado,POINT (-18.624 -49.616)
1,focos_brasil_ontem_hoje_ref.23d815fa-93d3-3562...,23d815fa-93d3-3562-98e6-caeea3e1120c,-49.612,-18.621,2020-09-25T17:10:00,AQUA_M-T,CAPINÓPOLIS,MINAS GERAIS,Brasil,3112604,31,33,2,0.0,1.0,Cerrado,POINT (-18.621 -49.612)
2,focos_brasil_ontem_hoje_ref.1f4db9ab-9adb-386d...,1f4db9ab-9adb-386d-a179-3d519b03f6c1,-46.861,-20.000,2020-09-25T17:10:00,AQUA_M-T,TAPIRA,MINAS GERAIS,Brasil,3168101,31,33,2,0.0,0.8,Cerrado,POINT (-20 -46.861)
3,focos_brasil_ontem_hoje_ref.cd97c8a3-f5d3-330e...,cd97c8a3-f5d3-330e-a0fa-64472972caae,-46.863,-19.989,2020-09-25T17:10:00,AQUA_M-T,TAPIRA,MINAS GERAIS,Brasil,3168101,31,33,2,0.0,0.7,Cerrado,POINT (-19.989 -46.863)
4,focos_brasil_ontem_hoje_ref.4101a440-ca4b-3805...,4101a440-ca4b-3805-a665-2c5fd62971b4,-46.846,-19.997,2020-09-25T17:10:00,AQUA_M-T,TAPIRA,MINAS GERAIS,Brasil,3168101,31,33,3,0.0,0.6,Cerrado,POINT (-19.997 -46.846)


In [14]:
def replace(match):
    data = {"á": "a", "ã" : "a", "Á": "A","Ã" : "A", "ç": "c", "ê": "e", "Ô":"O", "Ú":"U", "Ó":"O"}
    return data.get(match.group(0))

In [15]:
titulo = 'Queimadas - '+ lista[0][idx][0] 

map = fo.Map(location=[-23.5489, -46.6388], 
             title= titulo,
             zoom_start=5,
             width=900, 
             height=1200)
po = fo.FeatureGroup(name='Queimadas')

for row in range(len(df)):
    po.add_child(fo.Marker(location=[ df['latitude'][row], df['longitude'][row]],
                           popup=[re.sub(u'[^a-zA-Z0-9: ]', replace, df['municipio'][row]),
                                  re.sub(u'[^a-zA-Z0-9: ]', replace, df['estado'][row])],                           
                           icon=fo.Icon(color='red', icon='fire')
                          ))  
    
map.add_child(po)    